In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from parse import read_excel
from desdeo_tools.utilities import fast_non_dominated_sort_indices
import plotly.express as px

In [2]:
df = read_excel()

In [3]:
objectives = ["fearfulness", "human_aggression", "activity_playfulness", "cat_sociability", "human_sociability", "litterbox_issues", "excessive_grooming"]
directions = [1, 1, -1, -1, -1, 1, 1]

data = df[objectives].to_numpy() * directions

nadir, ideal = np.max(data, axis=0), np.min(data, axis=0)

In [4]:
fronts = fast_non_dominated_sort_indices(data)
pf_front = data[fronts[0]]
pf_data = df.loc[fronts[0]]

In [5]:
from collections import Counter

cats = dict(Counter(pf_data["BREEDGROUP"]))

cats

{'Siberian_and_Neva_Masquerade': 52,
 'Sphynx_and_Devon_Rex': 41,
 'European': 55,
 'Siamese_and_Balinese': 40,
 'Burmese': 28,
 'British': 28,
 'Oriental': 61,
 'Korat': 21,
 'House_cat': 211,
 'Sacred_Birman': 26,
 'Somali': 29,
 'Landrace_Cat_Shorthair': 263,
 'Maine_Coon': 48,
 'Other': 30,
 'Abyssinian': 44,
 'Turkish_Angora': 28,
 'Bengal': 40,
 'American_Curl': 42,
 'Landrace_Cat_Longhair': 50,
 'Norwegian_Forest_Cat': 29,
 'Ocicat': 45,
 'Russian_Blue': 12,
 'Turkish_Van': 10,
 'Ragdoll': 37,
 'Cornish_Rex': 25,
 'Persian_and_Exotic': 15}

In [6]:
pf_data = pf_data[["BREEDGROUP"] + objectives]
pf_data

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,Siberian_and_Neva_Masquerade,-0.592536,-0.376839,0.677554,0.247061,1.318160,-0.370580,-0.329714
3,Sphynx_and_Devon_Rex,-0.876063,-0.696218,1.255442,0.899894,0.475639,-0.514913,-0.259347
5,European,0.379262,-0.812642,0.343483,0.722727,-0.502684,-1.142007,-0.659040
7,Siamese_and_Balinese,-1.008219,-0.673436,1.421480,0.341644,0.850675,0.910663,-0.338755
10,Burmese,-0.813447,0.168712,1.363108,0.718478,0.661025,-0.545538,-0.033225
...,...,...,...,...,...,...,...,...
4309,Landrace_Cat_Shorthair,0.332575,-0.836463,0.957544,0.033670,0.774068,0.033412,-0.416599
4311,Korat,-1.196693,-0.315187,1.305526,0.822008,0.010866,-0.650696,-0.371469
4312,Korat,-1.563574,0.855251,1.400834,0.559283,-0.376932,-0.630812,-0.374776
4313,Korat,-1.515542,0.745532,1.412664,0.521732,-0.075399,-0.593704,-0.453170


In [7]:
# normalized_df=(df-df.min())/(df.max()-df.min())

pf_data[objectives] = ((pf_data[objectives] - pf_data[objectives].min()) / (pf_data[objectives].max() - pf_data[objectives].min()) * 100).round(decimals=2)

In [8]:
pf_data.round(decimals=2)

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,Siberian_and_Neva_Masquerade,26.30,15.78,70.30,66.20,78.83,20.56,10.14
3,Sphynx_and_Devon_Rex,19.47,9.59,79.65,78.12,67.07,18.00,10.92
5,European,49.71,7.34,64.89,74.89,53.42,6.88,6.50
7,Siamese_and_Balinese,16.28,10.03,82.34,67.93,72.30,43.28,10.04
10,Burmese,20.98,26.35,81.39,74.81,69.66,17.46,13.42
...,...,...,...,...,...,...,...,...
4309,Landrace_Cat_Shorthair,48.58,6.87,74.83,62.31,71.24,27.73,9.18
4311,Korat,11.74,16.97,80.46,76.70,60.59,15.60,9.68
4312,Korat,2.91,39.65,82.00,71.90,55.18,15.95,9.64
4313,Korat,4.06,37.52,82.19,71.22,59.38,16.61,8.78


In [9]:
# pf_data.to_csv("./data/pf_data.csv", index=False)

In [15]:
cats = dict(Counter(pf_data["BREEDGROUP"]))
cats

{'Siberian_and_Neva_Masquerade': 52,
 'Sphynx_and_Devon_Rex': 41,
 'European': 55,
 'Siamese_and_Balinese': 40,
 'Burmese': 28,
 'British': 28,
 'Oriental': 61,
 'Korat': 21,
 'House_cat': 211,
 'Sacred_Birman': 26,
 'Somali': 29,
 'Landrace_Cat_Shorthair': 263,
 'Maine_Coon': 48,
 'Other': 30,
 'Abyssinian': 44,
 'Turkish_Angora': 28,
 'Bengal': 40,
 'American_Curl': 42,
 'Landrace_Cat_Longhair': 50,
 'Norwegian_Forest_Cat': 29,
 'Ocicat': 45,
 'Russian_Blue': 12,
 'Turkish_Van': 10,
 'Ragdoll': 37,
 'Cornish_Rex': 25,
 'Persian_and_Exotic': 15}

In [17]:
cats_as_num = {f"{key}": i for (i, key) in enumerate(cats)}

In [18]:
cats_as_num

{'Siberian_and_Neva_Masquerade': 0,
 'Sphynx_and_Devon_Rex': 1,
 'European': 2,
 'Siamese_and_Balinese': 3,
 'Burmese': 4,
 'British': 5,
 'Oriental': 6,
 'Korat': 7,
 'House_cat': 8,
 'Sacred_Birman': 9,
 'Somali': 10,
 'Landrace_Cat_Shorthair': 11,
 'Maine_Coon': 12,
 'Other': 13,
 'Abyssinian': 14,
 'Turkish_Angora': 15,
 'Bengal': 16,
 'American_Curl': 17,
 'Landrace_Cat_Longhair': 18,
 'Norwegian_Forest_Cat': 19,
 'Ocicat': 20,
 'Russian_Blue': 21,
 'Turkish_Van': 22,
 'Ragdoll': 23,
 'Cornish_Rex': 24,
 'Persian_and_Exotic': 25}

In [21]:
pf_data['BREEDGROUP'] = pf_data['BREEDGROUP'].map(cats_as_num)

In [22]:
pf_data

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,0,26.30,15.78,70.30,66.20,78.83,20.56,10.14
3,1,19.47,9.59,79.65,78.12,67.07,18.00,10.92
5,2,49.71,7.34,64.89,74.89,53.42,6.88,6.50
7,3,16.28,10.03,82.34,67.93,72.30,43.28,10.04
10,4,20.98,26.35,81.39,74.81,69.66,17.46,13.42
...,...,...,...,...,...,...,...,...
4309,11,48.58,6.87,74.83,62.31,71.24,27.73,9.18
4311,7,11.74,16.97,80.46,76.70,60.59,15.60,9.68
4312,7,2.91,39.65,82.00,71.90,55.18,15.95,9.64
4313,7,4.06,37.52,82.19,71.22,59.38,16.61,8.78


In [26]:
pf_data

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,0,26.30,15.78,70.30,66.20,78.83,20.56,10.14
3,1,19.47,9.59,79.65,78.12,67.07,18.00,10.92
5,2,49.71,7.34,64.89,74.89,53.42,6.88,6.50
7,3,16.28,10.03,82.34,67.93,72.30,43.28,10.04
10,4,20.98,26.35,81.39,74.81,69.66,17.46,13.42
...,...,...,...,...,...,...,...,...
4309,11,48.58,6.87,74.83,62.31,71.24,27.73,9.18
4311,7,11.74,16.97,80.46,76.70,60.59,15.60,9.68
4312,7,2.91,39.65,82.00,71.90,55.18,15.95,9.64
4313,7,4.06,37.52,82.19,71.22,59.38,16.61,8.78


In [29]:
conversion = pd.DataFrame([cats_as_num])

In [30]:
conversion

,Siberian_and_Neva_Masquerade,Sphynx_and_Devon_Rex,European,Siamese_and_Balinese,Burmese,British,Oriental,Korat,House_cat,Sacred_Birman,...,Bengal,American_Curl,Landrace_Cat_Longhair,Norwegian_Forest_Cat,Ocicat,Russian_Blue,Turkish_Van,Ragdoll,Cornish_Rex,Persian_and_Exotic
0,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25


In [31]:
conversion.to_csv("./data/conversion.csv", index=False)